# Sprint ディープラーニングフレームワーク2

# 【問題1】公式チュートリアルモデルを分担して実行

* 選択課題：畳み込みニューラルネットワーク (Convolutional Neural Networks)
* 参照元url： https://colab.research.google.com/drive/1EjCLLDPX9vlTfT6alFIvyetG1OvmmD0Q?hl=ja#scrollTo=7Fv-UwP2S6gF

## TensorFlowのインポート

In [18]:
# !pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

## MNISTデータセットのダウンロードと準備

In [19]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

# (枚数、縦pixel、横pixel、カラーチャンネル数)
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# ピクセルの値を 0~1の間に正規化
train_images, test_images = train_images / 255.0, test_images / 255.0

In [26]:
test_images.shape

(10000, 28, 28, 1)

## 畳み込みの基礎部分の作成

* Conv2DとMaxPooling2Dレイヤーのスタック
* CNNはbatchサイズを無視して、shape(image_height, image_width, color_channels)のテンソルをとる。
* 今回のMNIST画像はグレースケールのため、color_channels=1、RGBの場合は3
* 入力フォーマット(28, 28, 1)を引数input_shapeとして最初のレイヤーに渡す。

In [20]:
model = models.Sequential()
# 最初の畳み込みレイヤー
# チャンネル数：32、抽出するストライド範囲、活性化関数、最初の層に必要な入力形式
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# 最初の隠れ層
# プーリング演算：(2, 2)はダウンスケールする係数。今回はそれぞれの次元で半分。
model.add(layers.MaxPooling2D((2, 2)))

# 2層目
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# 2番目の隠れ層
model.add(layers.MaxPooling2D((2, 2)))

# 3層目
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [21]:
# モデルのArchitectureを表示
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


## Denseレイヤーを追加

* 畳み込み後の出力（3, 3, 64）を、1つ以上のDenseレイヤーに入れて分類を実行する。
* Denseレイヤーは入力としてベクトル(1D)を取り入れる。
* そのため、畳み込み後の3D出力を1Dに平滑化してから、Denseレイヤーを追加する。
* 今回のMNISTは10個の出力クラスを持つため、最後のDenseレイヤーの出力を10にする。
* 最後にsoftmax関数を使用する。

In [22]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [23]:
# モデルのArchitectureを再度表示する。
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

## モデルのコンパイルと学習

畳み込みニューラルネットワークの計算グラフが完成したので、それを用いて最適化、損失関数、測定基準をコンパイル化（どのような学習方法をさせるか）する。

* 最適化方法：adam
* 損失関数：sparse_categorical_crossentropy
* 測定基準：accuracy

epochs数は5回分で学習させる。

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1426 - accuracy: 0.9571
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0487 - accuracy: 0.9849
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0331 - accuracy: 0.9895
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0261 - accuracy: 0.9918
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0198 - accuracy: 0.9938


## 学習済みモデルの評価

In [16]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

313/313 - 1s - loss: 0.0329 - accuracy: 0.9901


In [29]:
print(f'精度：{test_acc*100:.2f}%')

精度：99.01%


* 結果、99%以上のテスト精度を達成した。

# 【問題2】（アドバンス課題）様々な手法を実行

* 参照元url: https://github.com/tensorflow/models/tree/master/research/slim

# 【問題3】Iris（2値分類）をKerasで学習

* 隠れ層2枚、出力層1枚の多層パーセプトロン
* Kerasの基本的な流れ
  1. モデルを定義
  2. ネットワークを構築
  3. コンパイル
  4. 実行
----
**モデルを定義**
* 層の数：4
* 入力層：4変数
* 中間層（隠れ層①）：50ノード
* 中間層（隠れ層②）：100ノード
* 出力層（ラベル）：2変数（0 | 1)
* バッチ数：10
* epoch数：100（default=20）
* 活性化関数：relu
* 損失関数：categorical_crossentropy
* 最適化関数：Adam
* 評価関数：Accuracy（精度）

In [127]:
# ライブラリーとデータを読み出す。

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris


iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df = df.astype(float)
df['Species'] = iris.target
# 対象となる説明・目的変数（50～150行目）のみをスライスする。
df = df[50:150]

X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = df["Species"]

# 目的変数のラベルを0か1の二値に変換する。
y = y.replace(1, 0) # ラベル1を0に変換
y = y.replace(2, 1) # ラベル2を1に変換

X = np.array(X)
y = np.array(y)

# X = pd.DataFrame(iris.data[50:150, :], columns=iris.feature_names)
# y = pd.DataFrame(iris.target[50:150], columns=["Species"])

In [128]:
# 訓練データとテストデータへの分離
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0])

In [129]:
# 多層パーセプトロンの構築

model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(50, activation='relu', input_shape=(4, ), name="layer1"))
model.add(layers.Dense(100, activation='relu', name="layer2"))
model.add(layers.Dense(1, activation='softmax', name="output_layer"))

model.summary()

Model: "my_sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 50)                250       
_________________________________________________________________
layer2 (Dense)               (None, 100)               5100      
_________________________________________________________________
output_layer (Dense)         (None, 1)                 101       
Total params: 5,451
Trainable params: 5,451
Non-trainable params: 0
_________________________________________________________________


In [136]:
# 構築したmodelを用いcompile（どのように学習させるか）を設定する。
from keras.optimizers import Adam
model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [137]:
# 学習を実行する。
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=20,
                    validation_split=0.2
                    )

Epoch 1/20
6/6 [==============================] - 0s 21ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 2/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 3/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 4/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 5/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 6/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 - val_accuracy: 0.6000
Epoch 7/20
6/6 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.4833 - val_loss: 0.0000e+00 

In [123]:
model.evaluate(x = X_test, y = y_test)

1/1 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - accuracy: 0.4800


[0.0, 0.47999998927116394]